Overview...

This Python notebook demonsrates the _sliding window_ version of word count on a stream, based on a homework assignment in Konstantin's COMP 372 class. He's using this as a motivating example to understand imperative vs. functional programming (thinking in immutable terms).

This example is even interesting as a mutable example. In my solution, I have tried to avoid any Python tricks. We do use generators, which is Python's way of supporting _inifinitary_ thinking. The real work is in the `wc_on_generator()` method that processes the indefinite stream and--in the process--produces i a new infintary stream, where each stream is a dictionary (map) of the word counts within a given window.

This code will be packaged as a standalone Python program, but I like working things out in notebooks first and then turning them into more polished programs later. This can handle any stream. The user is prompted for `window_size` (the last number of lines to count words), `windows` (number of items to "take" from the resulting stream), and `min_count` (the minimum count to show, of great interest in digital humanities to avoid infrequent words).

It's not as beautiful as Scala but tries to be beautiful/idiomatic in a Pythonic sense. Enjoy!

In [1]:
from collections import deque
import urllib
import re

- We use a deque, which supports efficient access at either end of the deque. It is a built-in collections type in Python.
- urllib is only here to fetch some text from Project Gutenberg
- re is for regex and splitting the text into a stream of words

In [2]:
resource = urllib.request.urlopen("http://www.gutenberg.org/files/1524/1524-0.txt")
text = resource.read().decode("utf-8")
text = text[2000:] # skip unwanted header info (about 2000 chars) on Gutenberg's text.

In [3]:
word_pattern = re.compile("\w+")
words = re.findall(word_pattern, text)


We are mutable in two ways: 
- adding and removing items from the LRU cache (LRU here means least recent word within window)
- word counts for the current window


In [4]:
def wc_on_generator(word_generator, window_size):
    lru_word_cache = deque() # using deque but can also use fixed list/array in Python (next time)
    wc = {}
    for word in word_generator:
        wc[word] = wc.get(word, 0) + 1
        lru_word_cache.append(word)
        if len(lru_word_cache) > window_size:
            lru_item = lru_word_cache.popleft()
            wc[lru_item] = wc.get(lru_item, 1) - 1
        yield wc        

In [5]:
def show_counts(wc, min_count = 1):
    sorted_keys = list(wc.keys())
    sorted_keys.sort()
    report = ", ".join(["%s=%d" % (word, wc[word]) for word in sorted_keys if wc[word] >= min_count] )
    if len(report):
        return report
    else:
        return "No word occurred more than %d times in this window." % min_count

In [6]:
window_size = int(input("Window Size? "))
windows = int(input("Windows? "))
min_count = max(int(input("Minimum Count to Show? ")), 0)
wc_generator = wc_on_generator(words, window_size)
for window in range(0, windows):
    print("%d:" % window, show_counts(next(wc_generator), min_count))
    print()

Window Size? 100
Windows? 150
Minimum Count to Show? 2
0: No word occurred more than 2 times in this window.

1: No word occurred more than 2 times in this window.

2: No word occurred more than 2 times in this window.

3: No word occurred more than 2 times in this window.

4: No word occurred more than 2 times in this window.

5: No word occurred more than 2 times in this window.

6: No word occurred more than 2 times in this window.

7: Courtier=2

8: Courtier=2

9: Courtier=3

10: Courtier=3

11: Courtier=4

12: Courtier=4

13: Courtier=4

14: Courtier=4

15: Courtier=4, Officer=2

16: Courtier=4, Officer=2

17: Courtier=4, Officer=2

18: Courtier=4, Officer=2

19: Courtier=4, Officer=2

20: Courtier=5, Officer=2

21: Courtier=5, Officer=2

22: Courtier=5, Officer=2

23: Courtier=5, Officer=2

24: Courtier=5, Officer=2

25: Courtier=5, Officer=2

26: Courtier=5, Officer=2

27: Courtier=5, Officer=2

28: Courtier=6, Officer=2

29: A=2, Courtier=6, Officer=2

30: A=2, Courtier=6, Offi